# Consultas descriptivas del modelo

## Importación de librerías necesarias

In [1]:
import pandas as pd

## Importación de bases de datos

In [117]:
# Games
games = pd.read_json("../Data/corregida/r_games.json.gzip",compression="gzip",convert_dates=['release_date'],date_unit="ms")
# Reviews
reviews = pd.read_json("../Data/corregida/r_reviews.json.gzip",compression="gzip",convert_dates=["posted"],date_unit="ms")
# Items
items = pd.read_parquet("../Data/corregida/items.parquet.gzip")

## Creación de consultas

### PlayTimeGenre

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [50]:
# Se define la función PlayTimeGenre
def PlayTimeGenre(genero:str):
    # Se obtiene la base de juegos y se extraen los géneros por juego a través de un explode.
    g = games.explode("genres")
    # Se obtienen únicamente los juegos del género especificado.
    g = g[g["genres"] == genero]
    # Se obtiene la sumatoria de horas jugadas por juego a partir del dataframe de items.
    i = items.groupby("item_id")["playtime_forever"].sum()
    # Se obtiene un dataframe con las horas por juego y el año de lanzamiento.
    s = pd.merge(g,i,how = "inner",left_on="id",right_on="item_id")
    # Se agrupan las horas jugadas por año de lanzamiento.
    s = s.groupby(s["release_date"].dt.year)["playtime_forever"].sum()

    # se obtiene el año con más horas jugadas.
    año = s.idxmax()
    
    # Se genera la respuesta.
    respuesta = {f"Año de lanzamiento con más horas jugadas para el género {genero}":año}

    return respuesta


In [51]:
# Se prueba la fórmula
genero = "Action"
PlayTimeGenre(genero)

{'Año de lanzamiento con más horas jugadas para el género Action': 2012}

In [52]:
# Se prueba la fórmula
genero = "Free to Play"
PlayTimeGenre(genero)

{'Año de lanzamiento con más horas jugadas para el género Free to Play': 2013}

In [53]:
# Se prueba la fórmula
genero = "RPG"
PlayTimeGenre(genero)

{'Año de lanzamiento con más horas jugadas para el género RPG': 2011}

### UserForGenre

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [98]:
# Se define la función UserForGenre
def UserForGenre(genero:str):
    # Se obtiene la base de juegos y se extraen los géneros por juego a través de un explode.
    g = games.explode("genres")
    # Se obtienen únicamente los juegos del género especificado.
    g = g[g["genres"] == genero][["release_date","id"]]
    # Se obtiene la lista de id de juegos del género especificado.
    juegos = list(g["id"].unique())
    # Se obtienen los juegos de los usuarios que cumplen con la condición del género.
    i = items[items["item_id"].isin(juegos)]
    # Se unen en un mismo dataframe el año de lanzamiento y las horas jugadas.
    i = pd.merge(i,g,how="inner",left_on="item_id",right_on="id")
    # Se obtienen la sumatoria de horas por usuario.
    i = i.groupby(["user_id",i["release_date"].dt.year])["playtime_forever"].sum()
    # Se obtiene el id del jugador con más horas.
    player = i.idxmax()[0]
    # Se obtiene la información para el jugador identificado.
    player_data = i.loc[player]

    # Se estructura la respuesta: 
    response_data = {
        f"Usuario con más horas jugadas para el género {genero}":player,
        "Horas jugadas": [{"año":year,
                        "horas":playtime
                        }
                        for year,playtime in player_data.items()
                        ]
    }

    return response_data

In [99]:
# Se prueba la función.
genero = "Action"
UserForGenre(genero)

{'Usuario con más horas jugadas para el género Action': 'shinomegami',
 'Horas jugadas': [{'año': 1988, 'horas': 136},
  {'año': 1990, 'horas': 0},
  {'año': 1991, 'horas': 0},
  {'año': 1992, 'horas': 0},
  {'año': 1993, 'horas': 0},
  {'año': 1994, 'horas': 0},
  {'año': 1995, 'horas': 2},
  {'año': 1996, 'horas': 1161},
  {'año': 1997, 'horas': 99},
  {'año': 1998, 'horas': 3716},
  {'año': 1999, 'horas': 303},
  {'año': 2000, 'horas': 10775},
  {'año': 2001, 'horas': 2429},
  {'año': 2002, 'horas': 0},
  {'año': 2003, 'horas': 532756},
  {'año': 2004, 'horas': 1225},
  {'año': 2005, 'horas': 343},
  {'año': 2006, 'horas': 2180},
  {'año': 2007, 'horas': 68425},
  {'año': 2008, 'horas': 180},
  {'año': 2009, 'horas': 213225},
  {'año': 2010, 'horas': 9588},
  {'año': 2011, 'horas': 156697},
  {'año': 2012, 'horas': 20782},
  {'año': 2013, 'horas': 166380},
  {'año': 2014, 'horas': 180793},
  {'año': 2015, 'horas': 166348},
  {'año': 2016, 'horas': 42750},
  {'año': 2017, 'horas': 13

In [100]:
# Se prueba la función.
genero = "Free to Play"
UserForGenre(genero)

{'Usuario con más horas jugadas para el género Free to Play': 'idonothack',
 'Horas jugadas': [{'año': 2003, 'horas': 0},
  {'año': 2006, 'horas': 18},
  {'año': 2010, 'horas': 0},
  {'año': 2011, 'horas': 8},
  {'año': 2012, 'horas': 1794},
  {'año': 2013, 'horas': 29914},
  {'año': 2014, 'horas': 994},
  {'año': 2015, 'horas': 757025},
  {'año': 2016, 'horas': 16166},
  {'año': 2017, 'horas': 2322}]}

In [101]:
# Se prueba la función.
genero = "RPG"
UserForGenre(genero)

{'Usuario con más horas jugadas para el género RPG': 'shinomegami',
 'Horas jugadas': [{'año': 1999, 'horas': 377},
  {'año': 2000, 'horas': 0},
  {'año': 2001, 'horas': 0},
  {'año': 2002, 'horas': 0},
  {'año': 2003, 'horas': 530943},
  {'año': 2004, 'horas': 696},
  {'año': 2005, 'horas': 185},
  {'año': 2006, 'horas': 2023},
  {'año': 2007, 'horas': 135020},
  {'año': 2008, 'horas': 1305},
  {'año': 2009, 'horas': 1382},
  {'año': 2010, 'horas': 41407},
  {'año': 2011, 'horas': 17752},
  {'año': 2012, 'horas': 12819},
  {'año': 2013, 'horas': 107775},
  {'año': 2014, 'horas': 81939},
  {'año': 2015, 'horas': 111419},
  {'año': 2016, 'horas': 15550},
  {'año': 2017, 'horas': 0}]}

### UsersRecommend
def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [161]:
# Se define la función UsersRecommend
def UsersRecommend(año:int):
    # Se obtienen todos los juegos que tuvieron lanzamiento en el año especificado.
    g = games[games["release_date"].dt.year == año][["id","app_name"]]
    # Se obtienen los ID de los juegos 
    juegos = list(g["id"].unique())
    # Se obtienen los juegos con recomendación positiva para el año especificado. 
    r = reviews[reviews["item_id"].isin(juegos) & reviews["recommend"] == True & reviews["sentiment_analysis"].isin([1,2])][["item_id","recommend","sentiment_analysis"]]
    # Se obtiene la sumatoria de favorabilidad del juego.
    h = r.groupby("item_id")["sentiment_analysis"].sum()
    # Se unen los dataframes.
    g = pd.merge(g,h,how="inner",left_on="id",right_on="item_id")
    # Se obtienen los tres juegos más recomendados.
    recomendados = g.nlargest(3,"sentiment_analysis")
    # Se estructura la respuesta. 
    respuesta = {
        "Puesto 1":recomendados.iloc[0]["app_name"],
        "Puesto 2":recomendados.iloc[1]["app_name"],
        "Puesto 3":recomendados.iloc[2]["app_name"]
    }

    return respuesta

In [162]:
# Se prueba la función
año = 2015
UsersRecommend(año)

{'Puesto 1': 'Rocket League®',
 'Puesto 2': 'Grand Theft Auto V',
 'Puesto 3': 'Undertale'}

In [163]:
# Se prueba la función
año = 2016
UsersRecommend(año)

{'Puesto 1': 'Starbound',
 'Puesto 2': 'Heroes & Generals',
 'Puesto 3': "Don't Starve Together"}

In [167]:
# Se prueba la función
año = 2013
UsersRecommend(año)

{'Puesto 1': 'Rust', 'Puesto 2': 'DayZ', 'Puesto 3': 'Warframe'}

### UsersNotRecommend
def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [181]:
# Se define la función UsersNotRecommend
def UsersNotRecommend(año:int):
    # Se obtienen todos los juegos que tuvieron lanzamiento en el año especificado.
    g = games[games["release_date"].dt.year == año][["id","app_name"]]
    # Se obtienen los ID de los juegos 
    juegos = list(g["id"].unique())
    # Se obtienen los juegos con recomendación negativa para el año especificado. 
    r = reviews[(reviews["item_id"].isin(juegos)) & (reviews["recommend"] == False) & (reviews["sentiment_analysis"] == 0)][["item_id","recommend","sentiment_analysis"]]
    # Se obtiene el número de veces que el juego no fue recomendado.
    h = r.groupby("item_id")["sentiment_analysis"].count()
    # Se unen los dataframes.
    g = pd.merge(g,h,how="inner",left_on="id",right_on="item_id")
    # Se obtienen los tres juegos menos recomendados.
    no_recomendados = g.nlargest(3,"sentiment_analysis")
    # Se estructura la respuesta. 
    respuesta = {
        "Puesto 1":no_recomendados.iloc[0]["app_name"],
        "Puesto 2":no_recomendados.iloc[1]["app_name"],
        "Puesto 3":no_recomendados.iloc[2]["app_name"]
    }

    return respuesta

In [182]:
# Se prueba la función
año = 2015
UsersNotRecommend(año)

{'Puesto 1': 'Call of Duty®: Black Ops III',
 'Puesto 2': 'Mortal Kombat X',
 'Puesto 3': 'Dirty Bomb®'}

In [183]:
# Se prueba la función
año = 2011
UsersNotRecommend(año)

{'Puesto 1': 'APB Reloaded',
 'Puesto 2': 'No More Room in Hell',
 'Puesto 3': 'Terraria'}

In [184]:
# Se prueba la función
año = 2017
UsersNotRecommend(año)

{'Puesto 1': 'Unturned',
 'Puesto 2': 'Robocraft',
 'Puesto 3': 'ARK: Survival Evolved'}

### sentiment_analysis
def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [214]:
# Se define la función sentiment_analysis
def sentiment_analysis(año:int):
    # Se obtienen todas las reseñas para el año especificado:
    r = reviews[reviews["posted"].dt.year == año]
    # Se obtienen el número de comentario por cada categoria
    r = r.groupby("sentiment_analysis")["user_id"].count()
    # Se estructura la respuesta
    respuesta = {
    "Negative":r[0],
    "Neutral":r[1],
    "Positive":r[2]
    }

    return respuesta

In [215]:
# Se prueba la funcion
año = 2014
sentiment_analysis(año)

{'Negative': 1256, 'Neutral': 5023, 'Positive': 9847}

In [216]:
# Se prueba la funcion
año = 2015
sentiment_analysis(año)

{'Negative': 1312, 'Neutral': 4897, 'Positive': 7708}

In [217]:
# Se prueba la funcion
año = 2013
sentiment_analysis(año)

{'Negative': 281, 'Neutral': 1447, 'Positive': 3143}